<a href="https://colab.research.google.com/github/nicoRomeroCuruchet/DynamicProgramming/blob/main/testing_bary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
import numpy as np
import gymnasium as gym
from utils.utils import test_enviroment
from PolicyIteration import PolicyIteration

2024-09-30 17:39:12.217 | WARNING  | PolicyIteration:<module>:18 - CUDA is not available. Falling back to NumPy.


In [2]:
import airplane

glider = gym.make('ReducedSymmetricGliderPullout-v0')

bins_space = {
    "flight_path_angle": np.linspace(-np.pi, .001, 200,      dtype=np.float32),    # Flight Path Angle (γ)    (0)
    "airspeed_norm":     np.linspace(0.7, 4,       200,      dtype=np.float32),    # Air Speed         (V)    (1)
}

pi = PolicyIteration(
    env=glider, 
    bins_space=bins_space,
    action_space=np.linspace(-0.5, 1.0, 15, dtype=np.float32),
    gamma=0.99,
    theta=1e-3,
)


#pi.run()

2024-09-30 17:39:15.626 | INFO     | PolicyIteration:__init__:119 - Lower bounds: [-3.1415927  0.7      ]
2024-09-30 17:39:15.627 | INFO     | PolicyIteration:__init__:120 - Upper bounds: [1.e-03 4.e+00]
2024-09-30 17:39:15.629 | INFO     | PolicyIteration:__init__:132 - The action space is: [-0.5        -0.39285713 -0.2857143  -0.17857143 -0.07142857  0.03571429
  0.14285715  0.25        0.35714287  0.4642857   0.5714286   0.6785714
  0.78571427  0.89285713  1.        ]
2024-09-30 17:39:15.630 | INFO     | PolicyIteration:__init__:133 - Number of states: 40000
2024-09-30 17:39:15.631 | INFO     | PolicyIteration:__init__:134 - Total states:600000
2024-09-30 17:39:15.633 | INFO     | PolicyIteration:__init__:147 - Policy Iteration was correctly initialized.
2024-09-30 17:39:15.634 | INFO     | PolicyIteration:__init__:148 - The enviroment name is: TimeLimit


STALL_AIRSPEED: 27.331231856346


In [12]:
from utils.utils import get_optimal_action

with open(glider.__class__.__name__ + ".pkl", "rb") as f:
    pi: PolicyIteration = pickle.load(f)

dict_result = {}
for state in pi.states_space:
    state = np.array([-2.9777877,  1.7989212])
    initial_state = state.copy()
    glider.reset()
    glider.airplane.flight_path_angle = state[0]
    glider.airplane.airspeed_norm = state[1]
    total_reward = 0
    done = False
    episode_length = 0
    print(f"Initial state: {state}")
    while not done: #:and episode_length < 100:
        action = get_optimal_action(state, pi)
        state, reward, done, _, _ = glider.step(action)
        state = state[0]
        # check if our state is in the state space
        index = pi.triangulation.find_simplex(state)
        total_reward += reward*27.331231856346
        if done or index == -1: done = True
        episode_length += 1
        print(f"State: {state} - Reward: {total_reward} - Action: {action} - episode_length: {episode_length}")
    dict_result[tuple(initial_state)] = total_reward
    1/0
    #print(f"Initial state: {initial_state} - Total reward: {total_reward}")

c:\Users\nicor\anaconda3\envs\DynamicProgramming\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.airplane to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.airplane` for environment variables or `env.get_wrapper_attr('airplane')` that will search the reminding wrappers.
  logger.warn(


Initial state: [-2.9777877  1.7989212]
State: [-2.9777877  1.7989212] - Reward: -0.8017786141256547 - Action: -0.5 - episode_length: 1
State: [-2.9777877  1.7989212] - Reward: -1.603557228993414 - Action: -0.5 - episode_length: 2
State: [-2.9777877  1.7989212] - Reward: -2.4053358447822695 - Action: -0.5 - episode_length: 3
State: [-2.9777877  1.7989212] - Reward: -3.2071144616688745 - Action: -0.5 - episode_length: 4
State: [-2.9777877  1.7989212] - Reward: -4.008893079827423 - Action: -0.5 - episode_length: 5
State: [-2.9777877  1.7989212] - Reward: -4.810671699429539 - Action: -0.5 - episode_length: 6
State: [-2.9777877  1.7989212] - Reward: -5.612450320644154 - Action: -0.5 - episode_length: 7
State: [-2.9777877  1.7989212] - Reward: -6.414228943637402 - Action: -0.5 - episode_length: 8
State: [-2.9777877  1.7989212] - Reward: -7.216007568572512 - Action: -0.5 - episode_length: 9
State: [-2.9777877  1.7989212] - Reward: -8.017786195609704 - Action: -0.5 - episode_length: 10
State: 

KeyboardInterrupt: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Your dictionary with keys as coordinates (x, y) and values as the color intensity (z)
# Extracting the x, y and z values
x = [np.degrees(coord[0]) for coord in dict_result.keys() if coord[0] >= -np.pi/2]
#convert x to grad
y = [coord[1] for coord in dict_result.keys() if coord[0] >= -np.pi/2]
#convert y to Vs
keys_list = list(dict_result.keys())
values_list = list(dict_result.values())
z = [-v*20 for e,v in zip(keys_list, values_list) if e[0] >= -np.pi/2]

# Creating a 2D scatter plot
plt.scatter(y, x, c=z, cmap='turbo')  # Change 'viridis' to any other colormap you like
plt.colorbar(label='', shrink=0.8, )  # Add color bar for the z values

# Add labels and a title
plt.ylabel('Flight Path Angle (γ) [deg]')
plt.xlabel('V/Vs')

x_min, x_max = min(x), max(x)
y_min, y_max = min(y), max(y)


# Set the minor ticks for the grid, keeping the dense grid with smaller squares
ax = plt.gca()  # Get current axes
ax.set_yticks(np.linspace(x_min, x_max, 60), minor=True)  # 20 minor ticks for grid
ax.set_xticks(np.linspace(y_min, y_max, 60), minor=True)  # 20 minor ticks for grid

# Show the plot
plt.yticks(np.linspace(x_min, x_max, 6))  # Only 5 labels on x-axis
plt.xticks(np.linspace(y_min, y_max, 6))  # Only 5 labels on y-axis

# Enable the minor grid lines
ax.grid(which='minor', color='gray', linestyle='-', linewidth=0.5)

# Habilitar la cuadrícula en las marcas menores
ax.grid(which='minor', color='gray', linestyle='-', linewidth=0.5)

# Guardar la imagen sin borde blanco
plt.savefig('output.png', bbox_inches='tight')

# set size of the plot
plt.gcf().set_size_inches(9, 5)

plt.show()


# CartPoleEnv 

### Observation Space

The observation is a `ndarray` with shape `(4,)` with the values corresponding to the following positions and velocities:

| Num | Observation           | Min                 | Max               |
|-----|-----------------------|---------------------|-------------------|
| 0   | Cart Position         | -4.8                | 4.8               |
| 1   | Cart Velocity         | -Inf                | Inf               |
| 2   | Pole Angle            | ~ -0.418 rad (-24°) | ~ 0.418 rad (24°) |
| 3   | Pole Angular Velocity | -Inf                | Inf               |

### Action Space

The action is a `ndarray` with shape `(1,)` which can take values `{0, 1}` indicating the direction
of the fixed force the cart is pushed with.

- 0: Push cart to the left
- 1: Push cart to the right

In [ ]:
# Train cartpole environment:
from classic_control.cartpole import CartPoleEnv
# CartPole environment:
env = CartPoleEnv(sutton_barto_reward=True)
# position thresholds:
x_lim         = 2.4
theta_lim     = 0.418 
# velocity thresholds:
x_dot_lim     = 3.1
theta_dot_lim = 3.1

bins_space = {
    "x_space"         : np.linspace(-x_lim, x_lim, 10,  dtype=np.float32),                     # position space          (0)
    "x_dot_space"     : np.linspace(-x_dot_lim, x_dot_lim, 10,  dtype=np.float32),             # velocity space          (1)
    "theta_space"     : np.linspace(-theta_lim, theta_lim, 10, dtype=np.float32),              # angle space             (2)
    "theta_dot_space" : np.linspace(-theta_dot_lim, theta_dot_lim, 10, dtype=np.float32),      # angular velocity space  (3)
}

pi = PolicyIteration(
    env=env, 
    bins_space=bins_space,
    action_space=np.array([0, 1], dtype=np.int32),
    gamma=0.99,
    theta=1e-3
)

pi.run()

In [ ]:
# Test cartpole environment:

with open(env.__class__.__name__ + ".pkl", "rb") as f:
    pi = pickle.load(f)

test_enviroment(CartPoleEnv(sutton_barto_reward=True, render_mode="human"), pi)

## Observation Space

The observation is a `ndarray` with shape `(2,)` where the elements correspond to the following:

| Num | Observation                          | Min   | Max  | Unit         |
|-----|--------------------------------------|-------|------|--------------|
| 0   | position of the car along the x-axis | -1.2  | 0.6  | position (m) |
| 1   | velocity of the car                  | -0.07 | 0.07 | velocity (v) |

## Action Space

There are 3 discrete deterministic actions:

- 0: Accelerate to the left
- 1: Don't accelerate
- 2: Accelerate to the right


In [ ]:
from classic_control.continuous_mountain_car import Continuous_MountainCarEnv

env=Continuous_MountainCarEnv()

bins_space = {
    "x_space":     np.linspace(env.min_position, env.max_position, 100,      dtype=np.float32),    # position space    (0)
    "x_dot_space": np.linspace(-abs(env.max_speed), abs(env.max_speed), 100, dtype=np.float32),    # velocity space    (1)
}

pi = PolicyIteration(
    env=env, 
    bins_space=bins_space,
    action_space=np.linspace(-1.0, +1.0, 9, dtype=np.float32),
    gamma=0.99,
    theta=1e-3,
)
pi.run()

In [ ]:
# Test mountain car environment:
with open(env.__class__.__name__ + ".pkl", "rb") as f:
    pi: PolicyIteration = pickle.load(f)

test_enviroment(Continuous_MountainCarEnv(render_mode="human"), pi)